In [1]:
from model import PostgresStorage

pg = PostgresStorage.connect(host='vnkrtv.ru', user='vnkrtv', password='Hardpass1337', port=15432, dbname='vnkrtv')

In [2]:
corpus = list(pg.exec_query('SELECT text FROM posts LIMIT 3000', []))

In [3]:
import sys
!{sys.executable} -m pip install razdel

In [4]:
from typing import Generator, Iterable
import re

import nltk
import razdel 


class Tokenizer:
    remove_punctuation = re.compile(r'[^а-яА-ЯёЁ ]')

    @classmethod
    def tokenize(cls, text: str) -> Generator:
        return (cls.remove_punctuation.sub('', sentence.text.lower().strip())
                     for sentence in razdel.sentenize(text))


class TextProcessor:
    tokenizer = Tokenizer()

    @classmethod
    def get_sentences_gens(cls, texts: Iterable, remove_punctuation=True, remove_brackets=True) -> Generator:
        for text in texts:
            for sentence in cls.tokenizer.tokenize(text):
                yield sentence

    @classmethod
    def get_text_gen(cls, text_gen: Iterable) -> Generator:
        for sentence in cls.get_sentences_gens(text_gen):
            yield [_.text for _ in razdel.tokenize(sentence)]

    @classmethod
    def get_ngram_gen(cls, text_gen: Iterable, ngram_size: int = 3) -> Generator:
        for sentence in cls.get_sentences_gens(text_gen):
            yield [''.join(item) for item in nltk.ngrams(sentence, ngram_size)]


In [5]:
text = corpus[0][0]
list(TextProcessor.get_text_gen([text]))[0][:5], list(TextProcessor.get_ngram_gen([text]))[0][:5]

(['разработчики', 'объявили', 'о', 'появившейся', 'возможности'],
 ['раз', 'азр', 'зра', 'раб', 'або'])

In [7]:
import json
from typing import Generator, Dict, List

BEGIN = '__BEGIN__'
END = '__END__'
        
class WordsEncoder:
    counter: int
    word2int: Dict[str, int]
    int2word: Dict[int, str]

    def __init__(self, counter=None, word2int=None, int2word=None):
        if counter and word2int and int2word:
            self.counter = counter
            self.word2int = word2int
            self.int2word = int2word
                        
    def fit(self, text_corpus: Iterable):
        self.counter = 0
        self.word2int = {
            BEGIN: 0,
            END: -1
        }
        self.int2word = {
            0: BEGIN,
            -1: END
        }
        is_gen = isinstance(text_corpus, Generator)
        for sentence in text_corpus:
            for word in sentence:
                if word not in self.word2int:
                    self.counter += 1
                    self.word2int[word] = self.counter
                    self.int2word[self.counter] = word
                    
    def fit_encode(self, text_corpus: Iterable) -> List[List[int]]:
        self.fit(text_corpus)
        return self.encode_text_corpus(text_corpus)

    def encode_words_list(self, words_list: list) -> list:
        return [self.word2int[word] for word in words_list]

    def encode_text_corpus(self, text_corpus: list) -> list:
        """List of lists of words"""
        return [self.encode_words_list(words_list) for words_list in text_corpus]

    def encode_text_corpus_gen(self, text_corpus_gen: Generator) -> Generator:
        """List of lists of words"""
        return (self.encode_words_list(sentence.split()) for sentence in text_corpus_gen)

    def decode_codes_list(self, codes_list: list) -> list:
        return [self.int2word[code] for code in codes_list]

    def to_dict(self):
        """
        Returns the underlying data as a Python dict.
        """
        return {
            "counter": self.counter,
            "word2int": self.word2int,
            "int2word": self.int2word
        }

    def to_json(self):
        """
        Returns the underlying data as a JSON string.
        """
        return json.dumps(self.to_dict())

    @classmethod
    def from_dict(cls, obj):
        int2word = obj["int2word"]
        for key in int2word:
            int2word[int(key)] = int2word.pop(key)
        return cls(
            None,
            counter=obj["counter"],
            word2int=obj["word2int"],
            int2word=int2word
        )

    @classmethod
    def from_json(cls, json_str):
        return cls.from_dict(json.loads(json_str))

In [8]:
from typing import Iterable, Generator, Dict, List, Any
import re
import logging
import operator
import random
import bisect
import json
import copy


def accumulate(iterable: Iterable[Any], func=operator.add) -> Generator:
    it = iter(iterable)
    total = next(it)
    yield total
    for element in it:
        total = func(total, element)
        yield total

def compile_next(next_dict: Dict[tuple, dict]) -> List[list]:
    words = list(next_dict.keys())
    cff = list(accumulate(next_dict.values()))
    return [words, cff]

class EncodedChain:
    
    model: Dict[tuple, list]
    state_size: int

    def __init__(self, corpus: Iterable[list], state_size: int, model=None):
        self.state_size = state_size
        self.model = model or self.build(corpus, self.state_size)
        
    def get_random_init_state() -> tuple:
        return random.choice(list(self.model.keys()))

    def build(self, corpus, state_size) -> Dict[tuple, list]:
        model = {}

        for run in corpus:
            items = run + [ END ]
            for i in range(len(run) + 1 - state_size):
                state = tuple(items[i:i+state_size])
                follow = items[i+state_size]
                if follow != END:
                    follow = follow[-1]
                if state not in model:
                    model[state] = {}

                if follow not in model[state]:
                    model[state][follow] = 0

                model[state][follow] += 1
                
        model = { state: compile_next(next_dict) for (state, next_dict) in model.items() }
        return model

    def move(self, state: tuple) -> list:
        choices, cumdist = self.model.get(state, [None, None])
        if not choices:
            return END
        r = random.random() * cumdist[-1]
        selection = choices[bisect.bisect(cumdist, r)]
        return selection

    def gen(self, init_state: tuple = None) -> Generator:
        state = init_state
        while True:
            next_word = self.move(state)
            if next_word == END: 
                break
            
            yield next_word
            state = state[1:] + (state[-1][1:] + next_word, )

    def walk(self, init_state: tuple = None) -> list:
        return list(self.gen(init_state))

    def to_json(self) -> str:
        return json.dumps(list(self.model.items()))

    @classmethod
    def from_json(cls, json_thing: Any):
        if isinstance(json_thing, str):
            obj = json.loads(json_thing)
        else:
            obj = json_thing

        if isinstance(obj, list):
            rehydrated = dict((tuple(item[0]), item[1]) for item in obj)
        elif isinstance(obj, dict):
            rehydrated = obj
        else:
            raise ValueError("Object should be dict or list")

        state_size = len(list(rehydrated.keys())[0])

        inst = cls(None, state_size, rehydrated)
        return inst


In [9]:
train_corpus = list(TextProcessor.get_ngram_gen([text], 5))
encoder = WordsEncoder()
encoded_corpus = encoder.fit_encode(train_corpus)

In [10]:
chain = EncodedChain(train_corpus, state_size=2)

In [11]:
chain.model

{('разра', 'азраб'): [['о'], [1]],
 ('азраб', 'зрабо'): [['т'], [1]],
 ('зрабо', 'работ'): [['ч'], [1]],
 ('работ', 'аботч'): [['и'], [1]],
 ('аботч', 'ботчи'): [['к'], [1]],
 ('ботчи', 'отчик'): [['и'], [1]],
 ('отчик', 'тчики'): [[' '], [1]],
 ('тчики', 'чики '): [[' '], [1]],
 ('чики ', 'ики  '): [['о'], [1]],
 ('ики  ', 'ки  о'): [['б', ' '], [1, 2]],
 ('ки  о', 'и  об'): [['ъ'], [1]],
 ('и  об', '  объ'): [['я'], [1]],
 ('  объ', ' объя'): [['в'], [1]],
 (' объя', 'объяв'): [['и'], [1]],
 ('объяв', 'бъяви'): [['л'], [1]],
 ('бъяви', 'ъявил'): [['и'], [1]],
 ('ъявил', 'явили'): [[' '], [1]],
 ('явили', 'вили '): [['о'], [1]],
 ('вили ', 'или о'): [[' '], [1]],
 ('или о', 'ли о '): [['п'], [1]],
 ('ли о ', 'и о п'): [['о'], [1]],
 ('и о п', ' о по'): [['я'], [1]],
 (' о по', 'о поя'): [['в'], [1]],
 ('о поя', ' появ'): [['и'], [1]],
 (' появ', 'появи'): [['в'], [1]],
 ('появи', 'оявив'): [['ш'], [1]],
 ('оявив', 'явивш'): [['е'], [1]],
 ('явивш', 'вивше'): [['й'], [1]],
 ('вивше', '

In [12]:
''.join(chain.walk(('разра', 'азраб')))

'отчики  о непрерывности профессионального и образователи тоже получают сертификаты и дипломы от онлайнвиджета'

In [13]:
next(TextProcessor.get_ngram_gen(['Привет, Андрей!']))

['при', 'рив', 'иве', 'вет', 'ет ', 'т а', ' ан', 'анд', 'ндр', 'дре', 'рей']

In [63]:
class NgrammTextGenerator:

    chain: EncodedChain
    state_size: int
    ngram_size: int
    re_process: re.Pattern = re.compile(r'[^а-яА-ЯёЁ ]')

    def __init__(self,
                 state_size: int,
                 input_text: Iterable = None,
                 ngram_size: int = 3):
        self.state_size = state_size
        self.ngram_size = ngram_size
        if input_text:
            self.add_model(input_text)

    @classmethod
    def load(cls,
             state_size: int,
             ngram_size: int = 3):
        return cls(
            state_size=state_size,
            ngram_size=ngram_size)

    @classmethod
    def train(cls,
              train_text: Iterable,
              state_size: int,
              ngram_size: int = 3):
        return cls(
            input_text=train_text,
            state_size=state_size,
            ngram_size=ngram_size)

    def add_model(self, input_text: Iterable):
        train_corpus = list(TextProcessor.get_ngram_gen(input_text, self.ngram_size))
        self.chain = EncodedChain(train_corpus, self.state_size)
        logging.info(f'Add new model: {self}')

    def ngrams_split(self, sentence: str) -> List[str]:
        ngrams_list = next(TextProcessor.get_ngram_gen([sentence], self.ngram_size))
        return ngrams_list

    def ngrams_join(self, ngrams_list: List[str]) -> str:
        return ngrams_list[0][:-1] + ''.join([ngram[-1] for ngram in ngrams_list])

    def make_sentence(self, init_state: List[str], **kwargs) -> List[str]:
        tries = kwargs.get('tries', 10)
        max_words = kwargs.get('max_words', None)
        min_words = kwargs.get('min_words', None)

        for _ in range(tries):
            chars_list = self.chain.walk(tuple(init_state))
            if (max_words is not None and len(chars_list) > max_words) or (
                    min_words is not None and len(chars_list) < min_words):
                continue
            return chars_list
        return []

    def make_sentence_with_start(self, input_phrase: str, **kwargs) -> str:
        items_list = self.ngrams_split(input_phrase)
        items_count = len(items_list)

        if items_count == self.state_size:
            init_state = items_list

        elif self.state_size < items_count:
            init_state = items_list[-self.state_size:]
        else:
            return ''

        chars_list = self.make_sentence(init_state, **kwargs)
        return self.ngrams_join(items_list) + ''.join(chars_list)

    def make_sentences_for_t9(self,
                              beginning: str,
                              first_words_count: int = 1,
                              count: int = 30,
                              phrase_len: int = 5,
                              **kwargs) -> List[str]:
        phrases = set()
        for i in range(count):
            phrase = self.make_sentence_with_start(beginning, min_words=phrase_len, **kwargs)
            print(phrase)
            if phrase:
                words_list = phrase.split()
                if 1 < len(words_list) >= phrase_len:
                    phrases.add(" ".join(words_list[first_words_count:]))
        return list(phrases)

    def __repr__(self) -> str:
        return '<TextGenerator: state_size=%s, ngrams_size=%s>' % (
            self.state_size, self.ngram_size)


In [15]:
corpus = [row[0] for row in corpus]
len(corpus)

3000

In [64]:
model = NgrammTextGenerator.train(train_text=corpus,
                                  state_size=3,
                                  ngram_size=5)

In [97]:
model.make_sentences_for_t9('нейронные сети') 

нейронные сети к одному из супермаркета но явно помимо вышеуказанным тз
нейронные сети из нее теперь осталось на один хват
нейронные сети связано
нейронные сети одновременного и тд
нейронные сети структуру таблицы
нейронные сети и занимаюсь уже она не изучил вот это повторно собрать нужный гаджет в области  настоящим за недовольствие и просматривать а они казалось бы две вершин в каждом новой технологическом виде переключениями о проблема  лишь владела  теперь мне найти пассивные алгоритма  видно углубившись с ограничением  заключает в себе для нового место поэтому высокого уровня где ему нужно предзаказа как верхнего уровня что ближайшем будущем
нейронные сети  реализующие  элемента можно сказано в принцип объяснишь человека
нейронные сети будете породил целую кучу и включены к одному человека и понятных потоком заряжаем экстернализации можно изза требование блоба
нейронные сети насчет разную индуктивным  это достигала  тесла
нейронные сети для параметра   
нейронные сети нужно прост 

['сети замены нагрузка на сайта удалось устройства компьютерная хотя истинноваторскую дозу лекарства хостинг затем инструкции',
 'сети насчет разную индуктивным это достигала тесла',
 'сетимузыканта тупиковую крышку с головок закрытия',
 'сети будете породил целую кучу и включены к одному человека и понятных потоком заряжаем экстернализации можно изза требование блоба',
 'сети реализующие элемента можно сказано в принцип объяснишь человека',
 'сети дающая просьбе пользователь давиденко вячеслав основном не все еще имеют официальных двух крупные командный дух',
 'сети полной или иного продукте и его структуру конвертер модель только лет филиалов поэтому здесь еще немного человек должен задать новую формула предлагают ввести в файле прекрасно подойдет идентификацию там где ничего о скорости второй изменяется автомобильных версий на мой взгляд это мегабайт посвящается всяческих термоядерных итерацию плюсы компании никого на данных проекта его частью браузере',
 'сети к одному из супермарк

In [98]:
7.78 - 10.7

-2.919999999999999

In [101]:
model.chain.to_json()